In [1]:
import pandas as pd
pd.set_option('display.max_columns', 300)
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")

## Step 1: Read in hold out data, scalers, and best model

In [2]:
df2 = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [3]:
final_model = pickle.load( open( "model.pickle", "rb" ) )

final_model

LinearRegression()

In [4]:
with open('model_features.pickle', 'rb') as filename:
    model_features = pickle.load(filename)

model_features

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [6]:
df2.loc[df2['bathrooms']== 0, 'bathrooms'] = df2['bathrooms'].mean()
df2.loc[df2['bedrooms']== 0, 'bedrooms'] = df2['bedrooms'].mean()
# Updating mistyped data
df2.loc[df2['bedrooms'] >= 12, 'bedrooms'] = df2['bedrooms'].median()

In [7]:
# Isolating the date information in the string dtype date
df2['date'] = df2['date'].apply(lambda x: (x[:8]))

# converting date to float and then date format
df2['date'].astype(float)
df2['date2']= pd.to_datetime(df2['date'], format='%Y/%m/%d') # updating date column to date format in new column
df2.drop(columns = ['date'], inplace = True) # dropping object type date column 

In [8]:
df_orig_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15']

In [9]:
# generating new feature 1
df2['bath_per_bed'] = df2['bathrooms']/ df2['bedrooms']

# generating new feature 2
df2['bath_per_flr'] = df2['bathrooms']/ df2['floors']

# generating new feature 4
df2['age'] = 2020 - df2['yr_built']

# generating new feature 5
df2['use_prop'] = df2['sqft_living']/ df2['sqft_above']

# generating new feature 6
df2['sqft_per_flr'] = (df2['sqft_above'] + df2['sqft_basement'])/df2['floors']

# generating new feature 7
df2['outside_space'] = df2['sqft_lot'] - df2['sqft_per_flr']


In [10]:
df2['basement'] = df2['sqft_basement'].apply(lambda x: 1 if x > 0 else 0)

df2['renovated'] = df2['yr_renovated'].apply(lambda x: 1 if x > 0 else 0)

In [11]:
# binning the grade data to 3 levels of quality in a new feature design_grade
conditions = [
    df2['grade'] <= 4,
    df2['grade'] >= 10
]

choices = [
    'low_quality',
    'high_quality'
]
df2['design_grade'] = np.select(conditions, choices, default = 'ave_quality')


In [12]:
dummy1 = pd.get_dummies(df2['design_grade'])

In [13]:
df2.drop(columns = ['design_grade'], inplace = True)

In [14]:
# binning the condition data to 3 levels of house condition in a new feature status
conditions = [
    df2['condition'] == 1,
    df2['condition'] >= 4
]

choices = [
    'low',
    'high'
]
df2['status'] = np.select(conditions, choices, default = 'ave')

In [15]:
dummy2 = pd.get_dummies(df2['status'])
dummy2

,ave,high,low
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
4318,1,0,0
4319,1,0,0
4320,1,0,0
4321,1,0,0


In [16]:
df2.drop(columns = ['status'], inplace = True)

In [17]:
# new features from date
# The quarter of the year the house was sold
df2['sale_quarter'] = df2['date2'].dt.quarter

# The month the house was sold
df2['sale_month'] = df2['date2'].dt.month
df2.drop(columns = ['date2'], inplace = True)

In [18]:
df2 = pd.concat([df2, dummy1, dummy2], 1)
df2.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bath_per_bed,bath_per_flr,age,use_prop,sqft_per_flr,outside_space,basement,renovated,sale_quarter,sale_month,ave_quality,high_quality,low_quality,ave,high,low
0,1974300020,4.0,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0.625000,2.500,53,1.474026,2270.0,9230.0,1,0,3,8,1,0,0,1,0,0
1,1974300020,4.0,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0.625000,2.500,53,1.474026,2270.0,9230.0,1,0,1,2,1,0,0,1,0,0
2,3630020380,3.0,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576,0.833333,1.250,15,1.267241,735.0,1044.0,1,0,4,11,1,0,0,1,0,0
3,1771000290,3.0,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565,0.583333,1.750,44,1.242718,1280.0,14920.0,1,0,4,12,1,0,0,1,0,0
4,5126310470,4.0,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916,0.687500,1.375,15,1.000000,1415.0,6711.0,0,0,1,1,1,0,0,1,0,0


In [19]:
new_features = ['bath_per_bed', 'bath_per_flr', 'age', 'use_prop',
       'sqft_per_flr', 'outside_space', 'basement', 'renovated',
                'ave_quality', 'high_quality', 'low_quality',
       'ave', 'high', 'low']
df_new_features = df2[new_features]

In [20]:
df2.loc[df2['outside_space'] <=0, 'outside_space'] = 0

In [21]:
df2.drop(columns = ['id'], inplace = True)

In [22]:
data1 = df2[df_orig_features]._get_numeric_data()

In [23]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)
# here instantiating an object

poly_data = poly.fit_transform(data1)

poly_columns = poly.get_feature_names(data1.columns)

df_poly = pd.DataFrame(poly_data, columns = poly_columns)

In [24]:
df_poly.drop(columns = ['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15'], inplace = True)

In [25]:
df2 = pd.concat([df2, df_poly], axis = 1)

In [26]:
df2.shape

(4323, 205)

In [27]:
pd.set_option('display.max_rows', 300)
list(df2.columns)

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'bath_per_bed',
 'bath_per_flr',
 'age',
 'use_prop',
 'sqft_per_flr',
 'outside_space',
 'basement',
 'renovated',
 'sale_quarter',
 'sale_month',
 'ave_quality',
 'high_quality',
 'low_quality',
 'ave',
 'high',
 'low',
 'bedrooms^2',
 'bedrooms bathrooms',
 'bedrooms sqft_living',
 'bedrooms sqft_lot',
 'bedrooms floors',
 'bedrooms waterfront',
 'bedrooms view',
 'bedrooms condition',
 'bedrooms grade',
 'bedrooms sqft_above',
 'bedrooms sqft_basement',
 'bedrooms yr_built',
 'bedrooms yr_renovated',
 'bedrooms zipcode',
 'bedrooms lat',
 'bedrooms long',
 'bedrooms sqft_living15',
 'bedrooms sqft_lot15',
 'bathrooms^2',
 'bathrooms sqft_living',
 'bathrooms sqft_lot',
 'bathrooms floors',
 'bathrooms waterfront',
 'bathrooms view',
 'bathrooms 

## Step 3: Predict the holdout set

In [28]:
holdout_pred = final_model.predict(df2[model_features])
holdout_pred 

array([676266.62964249, 692230.69632339, 405897.88655853, ...,
       409113.85051346, 455610.95111847, 396392.95486069])

## Step 4: Export your predictions

In [29]:
pd.DataFrame(holdout_pred).to_csv('housing_preds_Lhamu.csv')